In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
df = pd.read_csv("../data/bangkok_traffy.csv")

In [3]:
df = df[["type", "comment", "province"]]
df = df[df['province'].str.contains('กรุงเทพ|bangkok',na=False,case=False)]
df = df.drop(["province"], axis=1)

df['type'].dropna(axis=0, inplace=True)
df['comment'].dropna(axis=0, inplace=True)

df['type'].fillna('', inplace=True) 
df['type'] = df['type'].astype(str) 
df['type'] = df['type'].str.strip('{}').apply(lambda x :[e for e in x.split(',') if e])
df = df.dropna().reset_index(drop=True)

In [4]:
df.isna().sum()

type       0
comment    0
dtype: int64

In [5]:
# import re
# import string
# import pythainlp
# from tqdm.auto import tqdm
# tqdm.pandas()

# def preprocessing_comment(comment):
#     # Normalize text
#     try:
#       comment = pythainlp.util.normalize(comment)
#     except:
#       print(comment)

#     # Tokenize text
#     tokens = pythainlp.word_tokenize(comment)
  
#     return comment

# df['comment_clean'] = df['comment'].progress_apply(preprocessing_comment)

In [6]:
df.head()

,type,comment
0,[],ไม่มีภาพ
1,[ความสะอาด],ขยะเยอะ
2,[สายไฟ],1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...
3,"[ถนน, สะพาน]",1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...
4,"[น้ำท่วม, ร้องเรียน]",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...


In [7]:
df["type"] = df["type"].apply(lambda x: np.nan if x == [] else x)
df = df.dropna().reset_index(drop=True)
df.head()

,type,comment
0,[ความสะอาด],ขยะเยอะ
1,[สายไฟ],1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...
2,"[ถนน, สะพาน]",1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...
3,"[น้ำท่วม, ร้องเรียน]",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...
4,"[ถนน, ร้องเรียน, น้ำท่วม, ความปลอดภัย]",1) พื้นที่นี้ขาดหน่วยงานรับผิดชอบ เนื่องจากไม่...


In [8]:
df.isna().sum()

type       0
comment    0
dtype: int64

In [9]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
transformed_data = mlb.fit_transform(df['type'])

dummy_df = pd.DataFrame(transformed_data, columns=[f'{label}' for label in mlb.classes_]).reset_index(drop=True)

In [10]:
# df.type

In [11]:
dummy_df.sum(axis=0).sort_values(ascending=False)

ถนน            76024
ทางเท้า        31251
แสงสว่าง       22506
ความปลอดภัย    20381
ความสะอาด      19690
น้ำท่วม        19006
กีดขวาง        15374
ท่อระบายน้ำ    13842
สะพาน          12111
จราจร          12030
สายไฟ           9583
เสียงรบกวน      8677
คลอง            8587
ต้นไม้          8008
ร้องเรียน       7250
ป้าย            6395
สัตว์จรจัด      4132
PM2.5           1116
สอบถาม          1109
เสนอแนะ          983
คนจรจัด          941
การเดินทาง       532
ห้องน้ำ          265
ป้ายจราจร        249
dtype: int64

In [12]:
def apply_1(x):
        if (len(x) == 1) and (["ร้องเรียน"] in x):
            x = ["other"]
        else:
            x = [elem for elem in x if elem not in ["ร้องเรียน"]]
        return x
def preprocess_type(df):
    df = df.copy()
    other_list = ["PM2.5", "สอบถาม", "เสนอแนะ", "คนจรจัด", "การเดินทาง", "ห้องน้ำ", "ป้ายจราจร"]
    df['type'] = df['type'].progress_apply(lambda x: [elem if elem not in other_list else 'other' for elem in x])
    df["type"] = df["type"].apply(lambda x: ["other"] if x == [] else x)
    df['type'] = df['type'].progress_apply(apply_1)
    df["type"] = df["type"].apply(lambda x: ["other"] if x == [] else x)
    return df

# tmp_df = preprocess_type(df)
tmp_df = df

In [13]:
tmp_df["type"] = tmp_df["type"].apply(lambda x: np.nan if x == [] else x)
tmp_df.isna().sum()

type       0
comment    0
dtype: int64

In [14]:
# tmp = "ร้องเรียน"
# filtered_df = tmp_df[tmp_df['type'].progress_apply(lambda x: any(tmp in elem for elem in x))]
# filtered_df.type.tolist()

In [15]:
tmp_df.sample(20)

,type,comment
188051,"[ความสะอาด, ร้องเรียน, ถนน]",ปัญหา : เมื่อวันที่ 08/04/65 พบพนักงานเก็บขนขย...
21341,[ถนน],ถนนนครชัยศรี เวลาฝนตก ถนนลื่นจะมีอุบัติเหตุลื่...
174622,"[ถนน, สอบถาม]",มีการสร้างสถานี จำหน่ายแก๊สในย่านชุมชน \n\nและ...
31746,"[ถนน, ต้นไม้]",บ้านที่อยู่ติดถนนปลูกต้นไม้นอกรั้วแล้วปล่อยให้...
137889,[ป้าย],ป้ายมีสติกเกอร์ติดเต็มไปหมด
125872,[ถนน],พื้นผิวถนนชำรุด
66907,"[ทางเท้า, ถนน]",1. ถนนทางเท้าขรุขระ คนชราเดินสะดุดบ่อยมากๆค่ะ
108521,[แสงสว่าง],ไฟส่องสว่างภายในที่พักผู้โดยสารชำรุด ขอความกรุ...
162029,[กีดขวาง],19 กพ. 66 นำรถปิคอัพ 2 คัน มาจอดบนทางเท้า ป...
78210,[ท่อระบายน้ำ],น้ำเริ่มเน่าท่อตัน


In [16]:
transformed_data = mlb.fit_transform(tmp_df['type'])

dummy_df = pd.DataFrame(transformed_data, columns=[f'type_{label}' for label in mlb.classes_]).reset_index(drop=True)
dummy_df.sum(axis=0).sort_values(ascending=False)

type_ถนน            76024
type_ทางเท้า        31251
type_แสงสว่าง       22506
type_ความปลอดภัย    20381
type_ความสะอาด      19690
type_น้ำท่วม        19006
type_กีดขวาง        15374
type_ท่อระบายน้ำ    13842
type_สะพาน          12111
type_จราจร          12030
type_สายไฟ           9583
type_เสียงรบกวน      8677
type_คลอง            8587
type_ต้นไม้          8008
type_ร้องเรียน       7250
type_ป้าย            6395
type_สัตว์จรจัด      4132
type_PM2.5           1116
type_สอบถาม          1109
type_เสนอแนะ          983
type_คนจรจัด          941
type_การเดินทาง       532
type_ห้องน้ำ          265
type_ป้ายจราจร        249
dtype: int64

In [19]:
[e[5:] for e  in dummy_df.sum(axis=0).sort_values(ascending=True).index.tolist()]

['ป้ายจราจร',
 'ห้องน้ำ',
 'การเดินทาง',
 'คนจรจัด',
 'เสนอแนะ',
 'สอบถาม',
 'PM2.5',
 'สัตว์จรจัด',
 'ป้าย',
 'ร้องเรียน',
 'ต้นไม้',
 'คลอง',
 'เสียงรบกวน',
 'สายไฟ',
 'จราจร',
 'สะพาน',
 'ท่อระบายน้ำ',
 'กีดขวาง',
 'น้ำท่วม',
 'ความสะอาด',
 'ความปลอดภัย',
 'แสงสว่าง',
 'ทางเท้า',
 'ถนน']

In [21]:
def apply_filter_type(x):
    order_list = ['ป้ายจราจร',
 'ห้องน้ำ',
 'การเดินทาง',
 'คนจรจัด',
 'เสนอแนะ',
 'สอบถาม',
 'PM2.5',
 'สัตว์จรจัด',
 'ป้าย',
 'ร้องเรียน',
 'ต้นไม้',
 'คลอง',
 'เสียงรบกวน',
 'สายไฟ',
 'จราจร',
 'สะพาน',
 'ท่อระบายน้ำ',
 'กีดขวาง',
 'น้ำท่วม',
 'ความสะอาด',
 'ความปลอดภัย',
 'แสงสว่าง',
 'ทางเท้า',
 'ถนน']
    for order in order_list:
        if order in x:
            return order
    print(x)
        
tmp_df_2 = tmp_df.copy()
tmp_df_2 = tmp_df_2["type"].apply(apply_filter_type)
tmp_df_2

0         ความสะอาด
1             สายไฟ
2             สะพาน
3         ร้องเรียน
4         ร้องเรียน
            ...    
205268          ถนน
205269        สายไฟ
205270          ถนน
205271      ทางเท้า
205272    ความสะอาด
Name: type, Length: 205273, dtype: object